# 通过one-api使用大模型

## 设置 one-api token

In [1]:
%%time

TOKEN="sk-W8fMtMdNWxNPxAf0F869DfB1Aa0c4bDf9263AbDfEa592d59"
HOST_URL="http://oneapi:3000"
MODEL_NAME="qwen2:1.5b"

%env TOKEN=$TOKEN
%env HOST_URL=$HOST_URL
%env MODEL_NAME=$MODEL_NAME

env: TOKEN=sk-W8fMtMdNWxNPxAf0F869DfB1Aa0c4bDf9263AbDfEa592d59
env: HOST_URL=http://oneapi:3000
env: MODEL_NAME=qwen2:1.5b
CPU times: user 282 μs, sys: 80 μs, total: 362 μs
Wall time: 357 μs


## 通过 Restful API 对话

In [2]:
%%time
%%bash

curl -s "$HOST_URL"/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $TOKEN" \
  -d "{
     \"model\": \"$MODEL_NAME\",
     \"messages\": [{\"role\": \"user\", \"content\": \"你谁啊\"}],
     \"temperature\": 0.7
   }" | jq .

{
  "id": "chatcmpl-57f23a3660984dceb9db3f4ab188e49b",
  "model": "qwen2:1.5b",
  "object": "chat.completion",
  "created": 1720683691,
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "我是来自阿里云的超大规模语言模型，我叫通义千问。很高兴为您服务！"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 11,
    "completion_tokens": 22,
    "total_tokens": 33
  }
}
CPU times: user 10.5 ms, sys: 7.55 ms, total: 18 ms
Wall time: 598 ms


## 通过 OpenAI API 对话

### 安装 openai 库

In [3]:
!pip install openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


### 同步方式问答

In [4]:
%%time

from openai import OpenAI

client = OpenAI(api_key=TOKEN, base_url=f"{HOST_URL}/v1")

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "你谁啊？"}],
)

print(response.choices[0].message.content)

我是一个大模型，来自阿里云。我叫通义千问，是阿里云研发的超大规模语言模型，也是国内首个通过机器学习技术实现的通用AI预训练语言模型。我的全名是通义千问·通义三地智能语言模型，由杭州小冰科技有限公司创作。我是中国首个人工智能语言处理类预训练语言模型，在全球范围内具有较高的知名度。
CPU times: user 344 ms, sys: 36.3 ms, total: 381 ms
Wall time: 1.86 s


### 流式问答

In [5]:
%%time

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "什么是贝叶斯主义？纯粹贝叶斯主义和实用贝叶斯主义有啥差别？"}],
    stream=True,
)

for chunk in response:
    print(chunk.choices[0].delta.content, end="")

贝叶斯主义是一种统计方法，用于推断一个概率模型中的参数。它是一种基于证据的统计推理过程，主要用于解决不确定性问题。

### 纯粹贝叶斯主义

**定义**: 纯粹贝叶斯主义是贝叶斯统计的一种特例或简化形式，其中所有先验知识（即初始信念）都为真。在这种情况下，我们使用纯粹贝叶斯概率来计算后验概率，而不是利用任何证据。

### 实用贝叶斯主义

**定义**: 实用贝叶斯主义是一种更加灵活的统计方法，允许基于证据推断参数。它认为先验知识可能不全然真实，因此实用贝叶斯方法可以使用更复杂的形式进行参数估计和不确定性分析。

### 差别

1. **先验概率与后验概率**: 在纯粹贝叶斯主义中，所有先验知识都视为真，而在实用贝叶斯主义中，先验概率可能包含不确定性和不完全信息。
2. **证据的作用**: 纯粹贝叶斯主义中的证据作用较小，因为它假设所有的先验知识都是正确的。在实用贝叶斯主义中，证据是核心工具，用于推断参数的后验分布。
3. **模型选择问题**: 在纯粹贝叶斯主义下，我们需要根据先验知识和数据来构建一个合理的概率模型。而在实用贝叶斯主义中，我们更关心如何使用证据优化现有模型。

简而言之，纯粹贝叶斯主义提供了一种简化统计推理的方法，而实用贝叶斯主义则在某种程度上更加灵活，可以根据实际数据和对先验知识的了解来调整参数估计的步骤。CPU times: user 231 ms, sys: 15.5 ms, total: 246 ms
Wall time: 5.47 s


### 使用云端模型

In [6]:
%%time

response = client.chat.completions.create(
    model="glm-4",
    messages=[{"role": "user", "content": "焦耳是用来度量什么的单位？"}],
    stream=True,
)

for chunk in response:
    print(chunk.choices[0].delta.content, end="")

焦耳（Joule，简称焦，符号J）是国际单位制（SI）中能量、功和热量的单位。它是用来度量物体由于力的作用而进行位移时所做的功，或者是热能的传递。具体来说：

1. 能量：焦耳用于度量物体的能量，无论是潜在的（如储存于电池中的电能）、还是动能（由于运动而具有的能量）。
2. 功：当力作用于物体并使其移动时，所做的工作可以用焦耳来度量。例如，1焦耳等于施加1牛顿的力使物体移动1米的距离所做的功。
3. 热量：在热力学中，焦耳也用于度量热量的传递，即热能的转移。

焦耳是一个导出单位，它可以根据不同的物理关系通过基本单位进行定义，如通过力的乘以位移（牛顿·米），或者电流的平方乘以电阻和时间（安培²·欧姆·秒），或者电荷乘以电压（库仑·伏特）。焦耳的名字是为了纪念英国物理学家詹姆斯·普雷斯科特·焦耳，他在能量守恒和热力学领域做出了重要贡献。CPU times: user 133 ms, sys: 14.7 ms, total: 148 ms
Wall time: 9.35 s
